In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from google.cloud import bigquery
import os
from google.cloud import bigquery
import glob


# pd.set_option('display.max_columns', None)

os.chdir('/home/jupyter')
!pwd
# # %cd 
# !pwd

import refinitiv.dataplatform as rdp

%run ./starfire/notebooks/refinitiv_credentials.ipynb

session = rdp.open_platform_session(
    APP_KEY,
    rdp.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
)



/home/jupyter


- AXJO starts in 1990 (official started in 2000)
- BSESN starts in 1980 (The base value of the SENSEX was taken as 100 on 1 April 1979 and its base year as 1978–79. 
- BVSP starts in 1980 (begun on 2 January 1968)
- FCHI (CAC) starts in 1987 (inaugurated on 23 June 1986)
- FTEU1 starts in 1990
- FTMIB starts in 1998 (as per wikipedia)
- FTSE starts in 1984 (Foundation 3 January 1984)
- GDAXI starts in 1988 (Foundation 1 July 1988)
- GSPTSE starts in 1980 (Foundation May 1, 2001; 23 years ago after replacing the TSX300 which started in 1945)
- HSI starts in 1965 (Foundation July 31, 1964)
- IBEX starts in 1992 (Foundation January 14, 1992)
- IRTS starts in 1996 (RTS Index, Introduced in September 1, 1995)
- KS200 starts in 1990 (??)
- MXX starts in 1988 (??)
- OMXS30 starts in 1987 (Foundation 1986)
- SSEC starts in  (Foundation 15 July 1991, launch date; 19 December 1990, base date))
- SSMI starts in 1989(introduced on 30 June 1988)
- TOPX starts in 1980 (?)
- TRXFLDAFPU, TRXFLDAZPU, TRXFLDLAPU start in 2000??
- TWII starts in 1976 (first published in 1967)

In [86]:
## Session needs to be closed and opened to avoid errors caused by ...who knows what.
rdp.close_session()
print('session is closed')
session = rdp.open_platform_session(
    APP_KEY,
    rdp.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
)
print('session is open')


ticker_list = ['.AXJO','.BSESN','.BVSP','.FTEU1','.FTSE','.GSPTSE','.KS200','.MXX','.SP500','.SSEC',
               '.TOPX','.TRXFLDAFPU','.TRXFLDAZPU','.TRXFLDLAPU', '.HSI', '.GDAXI', '.FCHI', '.TWII',\
               '.IBEX', '.FTMIB', '.SSMI', '.OMXS30', '.IRTS']

index = '.TWII'
sdate = '1967-01-01'
edate = '1974-12-31'

index_prices = rdp.get_data(
    index,
    fields = ['TR.CLOSEPRICE.Date','TR.CLOSEPRICE', 'TR.VOLUME'],
    parameters = {'SDate':sdate,'EDate':edate,'Curn':'USD'}
)
index_prices
print(index_prices)
rdp.close_session()
print('session is closed')
index_prices.to_csv(f'data_index_before_2000/{index.strip(".")}_{sdate}_{edate}.csv')


session is closed
session is open
     instrument                 Date  TR.CLOSEPRICE         TR.PriceOpen  \
0         .TWII  1967-01-05T00:00:00       3.601246  1969-12-30T00:00:00   
1         .TWII  1967-01-06T00:00:00       3.601246  1969-12-30T00:00:00   
2         .TWII  1967-01-07T00:00:00       3.609034  1970-01-05T00:00:00   
3         .TWII  1967-01-09T00:00:00       3.614486  1970-01-06T00:00:00   
4         .TWII  1967-01-10T00:00:00       3.614875  1970-01-07T00:00:00   
...         ...                  ...            ...                  ...   
2428      .TWII  1974-12-26T00:00:00       7.587617                 <NA>   
2429      .TWII  1974-12-27T00:00:00       7.353193                 <NA>   
2430      .TWII  1974-12-28T00:00:00       7.425234                 <NA>   
2431      .TWII  1974-12-30T00:00:00       7.517913                 <NA>   
2432      .TWII  1974-12-30T00:00:00       7.517913                 <NA>   

     TR.PriceOpen  TR.Volume  
0            None     

In [89]:

# Define the folder containing the CSV files
folder_path = 'data_index_before_2000/*.csv'

# Use glob to get all CSV file paths in the folder
csv_files = glob.glob(folder_path)

# Read and concatenate all CSV files into a single DataFrame
df_list = [pd.read_csv(file) for file in csv_files]
main_df = pd.concat(df_list, ignore_index=True)
main_df.columns = main_df.columns.str.replace('TR.', '', regex=False).str.lower()
# Drop rows with NaN in 'date' column
main_df = main_df.dropna(subset=['date'])

## Drop columns not needed and causing errors on BQ. I removed them from the refinitiv query
main_df = main_df.drop(columns=['priceopen', 'priceopen.1'])

# Extract the year and convert to integer
main_df['year'] = main_df['date'].str[:4].astype(int)

# Display the final concatenated DataFrame
print(main_df)

# Configure your BigQuery credentials and client
# Make sure you've set the GOOGLE_APPLICATION_CREDENTIALS environment variable
client = bigquery.Client()

# Define your BigQuery dataset and table
dataset_id = 'derived' 
table_id = 'equity_indices_xxx_2000'  

# Create a full table ID in the format of project.dataset.table
table_full_id = f'{client.project}.{dataset_id}.{table_id}'

# Upload the DataFrame to BigQuery
job = client.load_table_from_dataframe(main_df, table_full_id)

# Wait for the load job to complete
job.result()

print(f"Loaded {job.output_rows} rows into {table_full_id}.")


"""

The following query merged the two files (2000-2024 with xxx-1999), cleaning them up a bit:

CREATE OR REPLACE TABLE `starfire-410116.derived.equity_indices_all_times` AS
WITH aux AS (
  SELECT
    * EXCEPT (`unnamed: 0`)
  FROM `starfire-410116.derived.equity_indices_xxx_2000` t1
  LEFT JOIN(
    SELECT 
      * EXCEPT(count_deals, perc_deals, cum_perc)
    FROM `starfire-410116.derived.eqindx_country_dict`
  ) t2
  ON t1.instrument = t2.ticker
)
SELECT 
  * EXCEPT (`unnamed: 0`, priceopen, count_deals, perc_deals, cum_perc)
FROM `starfire-410116.derived.equity_indices_2000_2024`
UNION ALL SELECT * FROM aux 
"""

       unnamed: 0 instrument                 date  closeprice  volume  year
0               0     .BSESN  1990-01-01T00:00:00   46.517221     0.0  1990
1               1     .BSESN  1990-01-02T00:00:00   46.429167     0.0  1990
2               2     .BSESN  1990-01-03T00:00:00   45.187685     0.0  1990
3               3     .BSESN  1990-01-04T00:00:00   45.017751     0.0  1990
4               4     .BSESN  1990-01-05T00:00:00   45.887441     0.0  1990
...           ...        ...                  ...         ...     ...   ...
92168        1047      .IRTS  1999-12-27T00:00:00  156.613300     NaN  1999
92169        1048      .IRTS  1999-12-28T00:00:00  151.734600     NaN  1999
92170        1049      .IRTS  1999-12-29T00:00:00  147.280100     NaN  1999
92171        1050      .IRTS  1999-12-30T00:00:00  150.010600     NaN  1999
92172        1051      .IRTS  1999-12-31T00:00:00  175.262300     NaN  1999

[92172 rows x 6 columns]
Loaded 92172 rows into starfire-410116.derived.equity_indices_